In [1]:
# Load MNIST data 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import numpy as np

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
import tensorflow as tf 
sess = tf.InteractiveSession()

C:\Users\ychen\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [3]:
# Softmax Regression model 
# Nodes for input images and targe output classes 
# Used and modified by computation graph 
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
# Weight and bias
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# Initialize variables 
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
# Predicted Class and Loss Function 
y = tf.nn.softmax(tf.matmul(x,W) + b)
# Cross entropy loss 
cross_entropy = tf.reduce_mean( \
        -tf.reduce_sum(y_*tf.log(y), reduction_indices=[1]))
# reduce_sum: sum across classes
# reduce_mean: average over sums 

In [5]:
# Training 
# Steepest gradient descent, step length 0.5, cross entropy
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# Adding operations to computation graph 
# Compute gradients, compute parameter update step, apply update step to parameters

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
# Load 100 training examples each iteration 
# Run train_step, using feed_dict to replace placeholders 

In [6]:
# Evaluate the model 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean( \
    tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9191


In [4]:
# Convolutional Network 
# Weight initialization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Convolution and Pooling 
def conv2d(x, W):
    return tf.nn.conv2d(x, \
        W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], \
        strides=[1, 2, 2, 1], padding='SAME')

In [8]:
# First Convolutional Layer 
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
# Second Convolutional Layer 
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
# Densely Connected Layer 
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
# Dropout 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# Readout Layer 
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [13]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.82
step 200, training accuracy 0.88
step 300, training accuracy 0.94
step 400, training accuracy 0.9
step 500, training accuracy 0.96
step 600, training accuracy 0.92
step 700, training accuracy 0.94
step 800, training accuracy 0.98
step 900, training accuracy 1
test accuracy 0.9606


In [39]:
# Three-layer model for MNIST classification 
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_1 = weight_variable([784,100])
b_1 = bias_variable([100])
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

W_2 = weight_variable([100, 10])
b_2 = bias_variable([10])
y = tf.matmul(y_1, W_2) + b_2

# Initialize variables 
sess.run(tf.initialize_all_variables())

# Square loss
square_loss = tf.losses.mean_squared_error(y, y_)

In [40]:
# Training 
# Steepest gradient descent, step length 0.5, cross entropy
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(square_loss)
# Adding operations to computation graph 
# Compute gradients, compute parameter update step, apply update step to parameters

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# Evaluate the model 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean( \
    tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.916


In [8]:
# Conversion to Binary Labels
def convertBinary(oneHotEncoding):
    classes = np.where(oneHotEncoding == 1)[1]
    binEncoding = np.reshape(np.unpackbits(classes.astype(np.uint8)),(classes.shape[0],8))[:,4:]
    return binEncoding 

In [106]:
# 3 Layer with Binary Encoded Output
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 4])

W_1 = weight_variable([784,100])
b_1 = bias_variable([100])
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

W_2 = weight_variable([100, 4])
b_2 = bias_variable([4])
y = tf.matmul(y_1, W_2) + b_2

# Initialize variables 
sess.run(tf.initialize_all_variables())

# Cross entropy loss 
square_loss = tf.losses.mean_squared_error(y, y_)

In [107]:
# Training 
# Steepest gradient descent, step length 0.5, cross entropy
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(square_loss)
# Adding operations to computation graph 
# Compute gradients, compute parameter update step, apply update step to parameters

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: convertBinary(batch[1])})

# Evaluate the model 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean( \
    tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:convertBinary(mnist.test.labels)}))

0.6689


In [30]:
# Three-layer model for MNIST classification 
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_1 = weight_variable([784,1])
b_1 = bias_variable([1])
y_1 = tf.matmul(x, W_1) + b_1

W_2 = weight_variable([1, 10])
b_2 = bias_variable([10])
y = tf.nn.softmax(tf.matmul(y_1, W_2) + b_2)

# Initialize variables 
sess.run(tf.initialize_all_variables())

# Square loss
square_loss = tf.losses.mean_squared_error(y, y_)

In [31]:
# Training 
# Steepest gradient descent, step length 0.5, cross entropy
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(square_loss)
# Adding operations to computation graph 
# Compute gradients, compute parameter update step, apply update step to parameters

for i in range(10000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# Evaluate the model 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean( \
    tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))


0.4008
